**Credits goes to [discussion](https://www.kaggle.com/competitions/santa-2023/discussion/464694) shared by [toast-uz](https://www.kaggle.com/tatsuzoosawa)**

**Now you will love Rust** - i started learning rust for last week :)

In [ ]:
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
!source /root/.cargo/env
!/root/.cargo/bin/cargo new heuristic_transformer
%cd heuristic_transformer
! mkdir ./tools ./tools/in ./tools/out

In [ ]:

KAGGLE_DATA_DIR = '../../input/santa-2023'
HEURISTIC_INPUT_DIR = './tools/in'
HEURISTIC_OUTPUT_DIR = './tools/out'

import pandas as pd
import sys
import os
from ast import literal_eval
from itertools import groupby
import glob
import argparse

RED = '\033[1m\033[31m'
GREEN = '\033[1m\033[32m'
BLUE = '\033[1m\033[34m'
NORMAL = '\033[0m'

def make_input(puzzles, puzzle_info):    
    print(f'{GREEN}Writing {HEURISTIC_INPUT_DIR} ...{NORMAL}', end='', flush=True, file=sys.stderr)
    for _, puzzle in puzzles.iterrows():
        puzzle_id = puzzle['id']
        puzzle_type = puzzle['puzzle_type']
        solution_state = puzzle['solution_state'].split(';')
        facelet_map = {k: i for k, i, _ in [(k, i, len(list(g))) for i, (k, g) in enumerate(groupby(solution_state))]}
        solution_state = list(map(lambda k: facelet_map[k], solution_state))
        initial_state = list(map(lambda k: facelet_map[k], puzzle['initial_state'].split(';')))
        num_wildcards = puzzle['num_wildcards']
        allowed_moves = literal_eval(puzzle_info.loc[puzzle_type, 'allowed_moves'])
        allowed_moves = [(k, v) for k, v in allowed_moves.items()]
        assert len(solution_state) == len(initial_state), f'{RED}Solution state and initial state have different lengths{NORMAL}'
        print(f'{GREEN} {puzzle_id:04}.txt{NORMAL}', end='', flush=True, file=sys.stderr)
        # input_fileの書き込み
        with open(f'{HEURISTIC_INPUT_DIR}/{puzzle_id:04}.txt', 'w') as f:
            f.write(f'{len(solution_state)} {len(allowed_moves)} {num_wildcards}\n')
            f.write(f'{puzzle_type}\n')
            f.write(f'{" ".join(move_id for move_id, _ in allowed_moves)}\n')
            f.write(f'{" ".join(map(str, solution_state))}\n')
            f.write(f'{" ".join(map(str, initial_state))}\n')
            for _, perm in allowed_moves:
                f.write(f'{" ".join(map(str, perm))}\n')
    print(f'{GREEN}Done{NORMAL}', flush=True, file=sys.stderr)

def make_submission():
    cost = 0
    res = pd.DataFrame(columns=['id', 'moves'])
    for output_file in glob.glob(f'{HEURISTIC_OUTPUT_DIR}/*.txt'):
        id = int(os.path.splitext(os.path.basename(output_file))[0])
        with open(output_file, 'r') as f:
            moves = f.readline().rstrip()
        cost += len(moves.split("."))
        res = pd.concat([res, pd.DataFrame([[id, moves]], columns=['id', 'moves'])])
    res = res.sort_values('id')
    return res, cost

def make_output(submission):
    print(f'{GREEN}Writing {HEURISTIC_OUTPUT_DIR} ...{NORMAL}', end='', flush=True, file=sys.stderr)
    for _, row in submission.iterrows():
        id = row['id']
        moves = row['moves']
        with open(f'{HEURISTIC_OUTPUT_DIR}/{id:04}.txt', 'w') as f:
            f.write(f'{moves}\n')
    print(f'{GREEN}Done{NORMAL}', flush=True, file=sys.stderr)

In [ ]:
puzzles = pd.read_csv(f'{KAGGLE_DATA_DIR}/puzzles.csv')
puzzle_info = pd.read_csv(f'{KAGGLE_DATA_DIR}/puzzle_info.csv', index_col='puzzle_type')
make_input(puzzles, puzzle_info)

In [ ]:
submission = pd.read_csv("/kaggle/input/visualize-using-pythreejs/submission.csv")
make_output(submission)

In [ ]:
%%writefile Cargo.toml
[package]
name = "tools"
version = "0.1.0"
edition = "2021"

[dependencies]
proconio = { version = "=0.4.5", features = ["derive"] }
rustc-hash = "=1.1.0"

In [ ]:
%%writefile src/main.rs
use proconio::{input, source::auto::AutoSource};
use rustc_hash::FxHashMap as HashMap;

fn main() {
    let args: Vec<_> = std::env::args().collect();
    if args.len() != 3 {
        eprintln!("Usage: {} <input> <output>", args[0]);
        std::process::exit(1);
    }
    let e = Env::new(&args[1]);
    let a = Agent::new(&e, &args[2]);
    if !a.validate(&e) {
        std::process::exit(1);
    }
    eprintln!("Score = {}", a.compute_score());
}

#[allow(dead_code)]
#[derive(Debug)]
struct Env {
    n: usize, m: usize, w: usize,
    puzzle_type: String,
    move_ids: Vec<String>,
    solution_state: Vec<usize>,
    initial_state: Vec<usize>,
    allowed_moves: Vec<SparsePermutation>,
}

impl Env {
    fn new(input_file: &str) -> Self {
        let input_str = std::fs::read_to_string(input_file).unwrap();
        let input = AutoSource::from(&input_str[..]);
        input! {
            from input,
            n: usize, m: usize, w: usize,
            puzzle_type: String,
            move_ids: [String; m],
            solution_state: [usize; n],
            initial_state: [usize; n],
            allowed_moves: [[usize; n]; m],
        }
        let allowed_moves = allowed_moves.iter()
            .map(|x| SparsePermutation::new(x)).collect();
        Self { n, m, w, puzzle_type, move_ids,
            solution_state, initial_state, allowed_moves }
    }
}

struct Agent {
    moves: Vec<(usize, bool)>,  // (move_id, inverse_flag))    
}

impl Agent {
    fn new(e: &Env, output_file: &str) -> Self {
        let move_ids_hash: HashMap<String, usize> = e.move_ids.iter()
            .enumerate().map(|(i, s)| (s.clone(), i)).collect();
        let move_ids = std::fs::read_to_string(output_file)
            .unwrap().trim().to_string();
        let move_ids = move_ids.split('.');
        let mut moves = Vec::new();
        for move_id in move_ids {
            let (move_id, inverse_flag) =
                if let Some(move_id) = move_id.strip_prefix('-') {
                    (move_id, false) } else { (move_id, true) };
            let Some(&move_id) = move_ids_hash.get(&move_id.to_string()) else {
                eprintln!("Invalid move_id: {}", move_id);
                std::process::exit(1);
            };
            moves.push((move_id, inverse_flag));
        }
        Self { moves }
    }
    fn validate(&self, e: &Env) -> bool {
        let mut state = e.initial_state.clone();
        for &(move_id, inverse_flag) in &self.moves {
            let move_ = &e.allowed_moves[move_id];
            move_.apply_inplace(&mut state, inverse_flag);
        }
        let num_wrong_facelets =  state.wrong_metric(&e.solution_state);
        if num_wrong_facelets > e.w {
            eprintln!("Too many wrong facelets: {} > num_wildcards: {}", num_wrong_facelets, e.w);
            return false;
        }
        true
    }
    fn compute_score(&self) -> usize {
        self.moves.len()
    }
}

#[derive(Debug, Clone)]
pub struct SparsePermutation {
    from_to: Vec<(usize, usize)>,  // (from, to)
}

impl SparsePermutation {
    pub fn len(&self) -> usize { self.from_to.len() }
    pub fn new(perm: &[usize]) -> Self {
        let from_to = (0..perm.len())
            .filter(|&i| i != perm[i])  // make sparse
            .map(|i| (i, perm[i])).collect();
        Self { from_to }
    }
    pub fn apply(&self, x: &[usize], inverse_flag: bool) -> Vec<usize> {
        let mut res = x.to_vec();
        self.apply_inplace(&mut res, inverse_flag);
        res
    }
    pub fn apply_inplace(&self, x: &mut [usize], inverse_flag: bool) {
        if inverse_flag {
            let cache = (0..self.len())
                .map(|i| x[self.from_to[i].1]).collect::<Vec<_>>();
            for i in 0..self.len() { x[self.from_to[i].0] = cache[i]; }
        } else {
            let cache = (0..self.len())
                .map(|i| x[self.from_to[i].0]).collect::<Vec<_>>();
            for i in 0..self.len() { x[self.from_to[i].1] = cache[i]; }
        }
    }
}

trait WrongMetric {
    fn wrong_metric(&self, other: &Self) -> usize;
}

impl<T: PartialEq> WrongMetric for [T] {
    fn wrong_metric(&self, other: &Self) -> usize {
        (0..self.len()).filter(|&i| self[i] != other[i]).count()
    }
}

In [ ]:
! /root/.cargo/bin/cargo build

In [ ]:
!/root/.cargo/bin/cargo run --release ./tools/in/0000.txt ./tools/out/0000.txt

In [ ]:
with open("./run.sh","w") as f:
    for i, row in puzzles.iterrows():
        id = "{:04}.txt".format(row['id'])
        cmd = f"/root/.cargo/bin/cargo run --release ./tools/in/{id} ./tools/out/{id};\n"
        f.write(cmd)

In [ ]:
%%time
! sh run.sh

In [ ]:
submission, cost = make_submission()
print("Final Cost", cost)
submission.to_csv(f"./submission_{cost}.csv", index=False)